# Copied from https://www.kaggle.com/code/slawekbiel/fastai-inference

Just late baseline notebook, where you can easily test simple models on folds.

In [ ]:
from fastai.tabular.all import *
import ubiquant

### Load the data from feather and stick on GPU

In [ ]:
data_df = pd.read_feather('../input/ubiquant-trainfeather-32-bit/train32.feather')

ftrs = [f'f_{i}' for i in range(300)]
feature_tensor = torch.tensor(data_df[ftrs].to_numpy()).cuda()
target_tensor = torch.tensor(data_df.target.to_numpy()).cuda()

del data_df

In [ ]:
split1 = int(3141410 * 0.2)
split2 = int(3141410 * 0.4)
split3 = int(3141410 * 0.6)
split4 = int(3141410 * 0.8)

### Barebone lightweight dataloading

In [ ]:
class UbiquantDataset:
    def __init__(self, feature_tensor, targets):
        store_attr()
        self.n_inp = 2
    def __getitem__(self, idx):
        return torch.empty(0),self.feature_tensor[idx], self.targets[idx, None]
    
    def __len__(self):
        return len(self.feature_tensor)
    
class UbiDL(DataLoader):
    def __iter__(self):
        if self.shuffle:
            self.__idxs = torch.tensor(range(0,self.n))
        else:
            self.__idxs = torch.tensor(range(0,self.n))
        for batch_start in range(0, self.n, self.bs):
            if batch_start + self.bs > self.n and self.drop_last:
                return 
            indices = self.__idxs[batch_start:batch_start+self.bs]
            yield self.dataset[indices]

### A custom metric and loss function for training

In [ ]:
def pearson_coef(data):
    return data.corr()['target']['preds']

class CompMetric(AccumMetric):
    def __init__(self, val_df):
        super().__init__(None)
        self.val_df = val_df
        
    @property
    def name(self):
        return 'Pears'
        
    @property
    def value(self):
        preds = torch.cat(self.preds)
        val_df['preds'] = preds.cpu().numpy()
        return np.mean(self.val_df[['time_id', 'target', 'preds']].groupby('time_id').apply(pearson_coef))

In [ ]:
def pearson_loss(x, y):
    xd = x - x.mean()
    yd = y - y.mean()
    nom = (xd*yd).sum()
    denom = ((xd**2).sum() * (yd**2).sum()).sqrt()
    return 1 - nom / denom

### Use fast.ai Learner for trainig

In [ ]:
ds_train = UbiquantDataset(feature_tensor[:split1], target_tensor[:split1])
ds_val = UbiquantDataset(feature_tensor[split1:], target_tensor[split1:])

dls = DataLoaders.from_dsets(ds_train, ds_val, bs = 2048, dl_type=UbiDL, num_workers=0)

In [ ]:
model = TabularModel(emb_szs={}, n_cont=300, out_sz=1, layers = [69,64,32,16], act_cls=Swish()).cuda()
learn = Learner(dls, model, loss_func=pearson_loss, opt_func=Adam)
learn.fit(6, 0.002)
torch.save(learn.model, "fullmodel1.pth")

In [ ]:
ds_train = UbiquantDataset(feature_tensor[split1:split2], target_tensor[split1:split2])
ds_val = UbiquantDataset(feature_tensor[split2:], target_tensor[split2:])

dls = DataLoaders.from_dsets(ds_train, ds_val, bs = 2048, dl_type=UbiDL, num_workers=0)

In [ ]:
model = TabularModel(emb_szs={}, n_cont=300, out_sz=1, layers = [128,64,32,16], act_cls=Swish()).cuda()
learn = Learner(dls, model, loss_func=pearson_loss, opt_func=Adam)
learn.fit(6, 0.001)
torch.save(learn.model, "fullmodel2.pth")

In [ ]:
ds_train = UbiquantDataset(feature_tensor[split2:split3], target_tensor[split2:split3])
ds_val = UbiquantDataset(feature_tensor[split3:], target_tensor[split3:])

dls = DataLoaders.from_dsets(ds_train, ds_val, bs = 2048, dl_type=UbiDL, num_workers=0)

In [ ]:
model = TabularModel(emb_szs={}, n_cont=300, out_sz=1, layers = [128,64,32,16], act_cls=Swish()).cuda()
learn = Learner(dls, model, loss_func=pearson_loss, opt_func=Adam)
learn.fit(6, 0.002)
torch.save(learn.model, "fullmodel3.pth")

In [ ]:
ds_train = UbiquantDataset(feature_tensor[split3:split4], target_tensor[split3:split4])
ds_val = UbiquantDataset(feature_tensor[split4:], target_tensor[split4:])

dls = DataLoaders.from_dsets(ds_train, ds_val, bs = 2048, dl_type=UbiDL, num_workers=0)

In [ ]:
model = TabularModel(emb_szs={}, n_cont=300, out_sz=1, layers = [128,64,32,16], act_cls=Swish()).cuda()
learn = Learner(dls, model, loss_func=pearson_loss, opt_func=Adam)
learn.fit(6, 0.001)
torch.save(learn.model, "fullmodel4.pth")

In [ ]:
ds_train = UbiquantDataset(feature_tensor[split4:], target_tensor[split4:])
ds_val = UbiquantDataset(feature_tensor[:split1], target_tensor[:split1])

dls = DataLoaders.from_dsets(ds_train, ds_val, bs = 2048, dl_type=UbiDL, num_workers=0)

In [ ]:
model = TabularModel(emb_szs={}, n_cont=300, out_sz=1, layers = [128,64,32,16], act_cls=Swish()).cuda()
learn = Learner(dls, model, loss_func=pearson_loss, opt_func=Adam)
learn.fit(6, 0.002)
torch.save(learn.model, "fullmodel5.pth")

# Create submission

In [ ]:
model1 = torch.load('./fullmodel1.pth').cuda().eval()
model2 = torch.load('./fullmodel2.pth').cuda().eval()
model3 = torch.load('./fullmodel3.pth').cuda().eval()
model4 = torch.load('./fullmodel4.pth').cuda().eval()
model5 = torch.load('./fullmodel5.pth').cuda().eval()

ftrs = [f'f_{i}' for i in range(300)]
env = ubiquant.make_env()  
iter_test = env.iter_test()
for (test_df, sub_df) in iter_test:
    data = torch.tensor(test_df[ftrs].to_numpy(), dtype=torch.float).cuda()
    with torch.no_grad():
        pred1 = model1([], data)
        pred2 = model2([], data)
        pred3 = model3([], data)
        pred4 = model4([], data)
        pred5 = model5([], data)

        preds = (pred1 + pred2 + pred3 + pred4 + pred5) / 5
    sub_df['target'] = preds.view(-1).cpu().numpy()
    env.predict(sub_df) 